In [75]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

print(tf.__version__)


2.8.0


In [68]:
# def convnet_model_():
#     vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
#     vgg.trainable = False
#     model = tf.keras.Sequential([
#         #VGG16
#         vgg,
#         #GlobalAveragePooling2D
#         tf.keras.layers.GlobalAveragePooling2D(),
#         #Dense 4096 relu
#         tf.keras.layers.Dense(4096, activation='relu'),
#         # #Dropout 0.5
#         # tf.keras.layers.Dropout(0.5),
#         # #Dense 4096 relu
#         # tf.keras.layers.Dense(4096, activation='relu'),
#         #Dropout 0.5
#         tf.keras.layers.Dropout(0.5),
#         #L2 normalization
#         tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
#     ])

#     return model

def convnet_model_():
    URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
    feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224,3))
    feature_extractor.trainable = False
    model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(4096, activation='relu'),
        #L2 normalization
        tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
    ])

    return model

def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


In [69]:
deep_rank_model = deep_rank_model()

for layer in deep_rank_model.layers:
    print (layer.name, layer.output_shape)

model_path = "./deep_ranking"



input_20 [(None, 224, 224, 3)]
input_21 [(None, 224, 224, 3)]
conv2d_16 (None, 14, 14, 96)
conv2d_17 (None, 7, 7, 96)
max_pooling2d_16 (None, 4, 4, 96)
max_pooling2d_17 (None, 4, 4, 96)
flatten_16 (None, 1536)
flatten_17 (None, 1536)
keras_layer_5_input [(None, 224, 224, 3)]
lambda_33 (None, 1536)
lambda_34 (None, 1536)
keras_layer_5 (None, 1280)
concatenate_16 (None, 3072)
lambda_32 (None, 1280)
concatenate_17 (None, 4352)
dense_16 (None, 4096)
lambda_35 (None, 4096)


In [70]:
batch_size = 8
batch_size *= 3


In [71]:
_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    loss =  tf.convert_to_tensor(0,dtype=tf.float32)
    g = tf.constant(1.0, shape=[1], dtype=tf.float32)
    for i in range(0,batch_size,3):
        try:
            q_embedding = y_pred[i+0]
            p_embedding =  y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p =  K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = (loss + g + D_q_p - D_q_n )            
        except:
            continue
    loss = loss/(batch_size/3)
    zero = tf.constant(0.0, shape=[1], dtype=tf.float32)
    return tf.maximum(loss,zero)



In [72]:
pred = deep_rank_model.predict((np.random.rand(10,224,224,3),np.random.rand(10,224,224,3),np.random.rand(10,224,224,3)))
print(pred.shape)
_loss_tensor(None,pred).numpy()

(10, 4096)


array([0.37713522], dtype=float32)

In [73]:
#repeat
gen1 = ImageDataGenerator(rescale=1./255)
gen1 = gen1.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)
gen2 = ImageDataGenerator(rescale=1./255)
gen2 = gen2.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)
gen3 = ImageDataGenerator(rescale=1./255)
gen3 = gen3.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)

class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1, input_gen2, target_gen):
        self.gen1 = input_gen1
        self.gen2 = input_gen2
        self.gen3 = target_gen

        assert len(input_gen1) == len(input_gen2) == len(target_gen)

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
        x1 = self.gen1[i]
        x2 = self.gen2[i]
        y = self.gen3[i]

        return [x1, x2, y], y

    def on_epoch_end(self):
        self.gen1.on_epoch_end()
        self.gen2.on_epoch_end()
        self.gen3.on_epoch_end()

my_gen = JoinedGen(gen1, gen2, gen3)


Found 3000 images belonging to 1 classes.
Found 3000 images belonging to 1 classes.
Found 3000 images belonging to 1 classes.


In [74]:
#deep_rank_model.load_weights('deepranking.h5')
deep_rank_model.compile(loss=_loss_tensor, optimizer='adam')


# train_steps_per_epoch = int((59520)/batch_size)
train_steps_per_epoch = 125
train_epochs = 25
deep_rank_model.fit_generator(my_gen,
                        steps_per_epoch=train_steps_per_epoch,
                        epochs=train_epochs
                        )

model_path = "deepranking.h5"
deep_rank_model.save_weights(model_path)
#f = open('deepranking.json','w')
#f.write(deep_rank_model.to_json())
#f.close()

C:\Users\Timothe\AppData\Local\Temp/ipykernel_3108/1073118864.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  deep_rank_model.fit_generator(my_gen,


Epoch 1/25
125/125 [==============================] - 83s 626ms/step - loss: 0.9989
Epoch 2/25
125/125 [==============================] - 88s 705ms/step - loss: 0.9543
Epoch 3/25
125/125 [==============================] - 87s 694ms/step - loss: 0.8824
Epoch 4/25
 86/125 [===================>..........] - ETA: 27s - loss: 0.8011

KeyboardInterrupt: 